In [4]:
# Necessary imports
%load_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import pandas as pd

from moge.network.heterogeneous_network import HeterogeneousNetwork

from sklearn.model_selection import ParameterGrid, ParameterSampler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pickle

# READ
with open('moge/data/LMN_future_recall/TRAIN/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.train.pickle', 'rb') as file:
    network = pickle.load(file)
    
with open('moge/data/LMN_future_recall/TEST/Interactions_Affinity/LMN_mirtarbase_biogrid_starbase_lncrna2target_lncrinter.test.pickle', 'rb') as file:
    network_val = pickle.load(file)

# Set parameter space

In [3]:
parameters = {
    "d": [128, 512],
    "lr": [0.001, 0.0005],
#     "margin": [0.1, 0.2, 0.5],
#     "compression_func": ["sqrt", "log"],
    "negative_sampling_ratio": [2.0, 5.0, 8.0],
#     "directed_proba": [0.5, 0.8],
    "max_length": [1400, 1800],
    "truncating": ["random", "post"],
    
    "conv1_kernel_size": [6, 12, 18],
    "max1_pool_size": [3, 4, 6],
    "conv2_kernel_size": [None, 2, 4, 6],
    "max2_pool_size": [2, 4, 6],
    "lstm_unit_size": [160, 320],
    "dense1_unit_size": [256, 1024],
    "dense2_unit_size": [None, 256, 512],
}

# Train Model

In [4]:
from moge.embedding.siamese_graph_embedding import SiameseGraphEmbedding, SiameseTripletGraphEmbedding

siamese = SiameseGraphEmbedding(batch_size=925, epochs=1, verbose=False)

# siamese = SiameseTripletGraphEmbedding(batch_size=256, epochs=1, verbose=False)


Using TensorFlow backend.


In [ ]:
best_score = float("inf")
X_params = []
y = []

for g in ParameterSampler(parameters, n_iter=300):
    print(g)
    siamese.set_params(**g)
    
    try:
        siamese.learn_embedding(network, network_val=network_val, multi_gpu=True, 
                            subsample=False, n_steps=1000, validation_steps=None, 
                            validation_make_data=False, rebuild_model=True,
                            seed=42)
    except Exception as e:
        print("Failed!!!", e)
        continue
    except KeyboardInterrupt as e:
        continue
    
    current_score = siamese.hist.history['val_loss'][-1]
    X_params.append(g)
    y.append(current_score)
    
    if current_score < best_score:
        best_score = current_score
        best_grid = g
        best_history = siamese.hist.history
        print("Score:", best_score, "\n")
    


{'truncating': 'random', 'negative_sampling_ratio': 8.0, 'max_length': 1400, 'max2_pool_size': 4, 'max1_pool_size': 6, 'lstm_unit_size': 320, 'lr': 0.0005, 'dense2_unit_size': 512, 'dense1_unit_size': 1024, 'd': 128, 'conv2_kernel_size': None, 'conv1_kernel_size': 12}
Epoch 1/1
6783/6783 [==============================] - 1762s 260ms/step - loss: 0.0809 - val_loss: 0.1838
Score: 0.18375178881381687 

{'truncating': 'post', 'negative_sampling_ratio': 5.0, 'max_length': 1400, 'max2_pool_size': 2, 'max1_pool_size': 6, 'lstm_unit_size': 320, 'lr': 0.0005, 'dense2_unit_size': 256, 'dense1_unit_size': 1024, 'd': 512, 'conv2_kernel_size': None, 'conv1_kernel_size': 6}
Epoch 1/1
6783/6783 [==============================] - 1797s 265ms/step - loss: 0.0835 - val_loss: 0.1891
Epoch 1/1
{'truncating': 'random', 'negative_sampling_ratio': 2.0, 'max_length': 1800, 'max2_pool_size': 6, 'max1_pool_size': 4, 'lstm_unit_size': 160, 'lr': 0.001, 'dense2_unit_size': 512, 'dense1_unit_size': 256, 'd': 128,

In [ ]:
print(best_score)

In [ ]:
print(best_grid)

In [ ]:
print(best_history)